# 0. Выгрузка данных о статьях авторов

**Парсинг данных eLibrary для проекта "Паттерны коллаборации в Российском социологическом сообществе" - id авторов**

В этом файле производится выгрузка данных eLibrary для проекта Лаборатории по гранту РНФ "Паттерны коллаборации в Российском социологическом сообществе". Осуществляется импорт необходимых библиотек, загружаются id авторов, данные о которых необходимо выгрузить, производится выгрузка кратких данных о всех статьях этих авторов из API Elibrary.

Основной код выгрузки к `API-011` Elibrary был подготовлен Ликой Капустиной в сентябре-октябре 2022 года. Цель выгрузки - выгрузить краткие справочные данные о статьях всех уникальных авторов, встречающихся в основном массиве данных (но не все данные по статьям).

Собранные нами данные не использовались для дообновления финальных данных проекта, но код в этом ноутбуке может быть полезен для других исследователей.

**Результат выгрузки:**

Файл `all_final_authors_with_papers.csv`, содержащие в себе информацию о всех статьях всех авторов, присутствовавших в 8 колонках наших изначальных данных.

## Подготовка к работе

### Загрузка необходимых библиотек

In [5]:
# !pip install xmltodict # при необходимости!
from numpy import fromstring # приведение данных к строке
import numpy as np
import requests # модуль для запросов к данным по ссылкам
import xmltodict # работа с xml
import pandas as pd # модуль для работы с таблицами
import tqdm # модуль для работы с tqdm
from time import sleep # импортируем функцию сна 
import warnings # модуль с предупреждениями
warnings.filterwarnings("ignore") # выбираем игнорировать предупреждения
import xml.etree.ElementTree as ET # работа с xml
from xml.etree.ElementTree import parse # работа с xml
from urllib.request import urlopen # работа с запросами
import pickle # формат данных pickle

### Проверка ip-адреса

По условиям договора с Elibrary, выгрузка данных могла производиться только по рабочему ip-адресу Дарьи Васильевны в корпусе Вышки на Покровском бульваре, поэтому перед выгрузкой данных осуществлялось подключение к ip-адресу Дарьи Васильевны. В ячейке ниже я проверяю ip-address

In [3]:
import http.client # модуль для работы с ip
conn = http.client.HTTPConnection("ifconfig.me")
conn.request("GET", "/ip")
conn.getresponse().read() # ip-адрес корректный

b'92.242.58.41'

### Загрузка датафрейма с данными по публикациям

Сервис `011` API Elibrary собрать информацию об авторах с ID. В коде ниже происходит следующее: 

1. Мы загружаем данные по статьям, которые собрали предварительно; 
2. Получаем список с id (внутренними id Elibrary) авторов в наших данных;
3. Используем написанный код для получения данных из сервиса `011` API Elibrary и получаем `pandas.DataFrame` с id и другой справочной информацией о всех статьях этих авторов;
4. Сохраняем полученный набор данных в файл `all_final_authors_with_papers.csv`

In [15]:
# 1. Загружаем данные по статьям
df = pd.read_excel('FINAL_CORRECT_SOCIOLOGISTS.xlsx')
df.head()

,Column1,id,genreId,author_1_id,author_1_affiliation_id,author_1_ru last name,author_1_ru initials,author_1_ru affiliation,author_1_eng last name,author_1_eng initials,...,genre_id,cited,corecited,parent_id,source_id,isFT,isNEW,link,url,ref
0,0,15485609,4,0.0,"[['7113'], ['7113']]",Кийков,Алексей Владимирович,['Белгородский государственный технологический...,Kiykov,Aleksey Vladimirovich,...,4,3,1.0,33649315,NaN,1,0,http://elibrary.ru/item.asp?id=15485609,none,"Кийков, А.В. СОСТОЯНИЕ И ТЕНДЕНЦИИ СЕТЕВОГО ВЗ..."
1,1,15488335,4,675896.0,"['none', 'none']",Салагаев,А.,none,none,none,...,4,0,0.0,33649638,NaN,1,1,http://elibrary.ru/item.asp?id=15488335,NaN,"Салагаев, А. КУЛЬТУРНАЯ И РЕЛИГИОЗНАЯ СИТУАЦИЯ..."
2,2,15488853,4,820606.0,"[['340'], 'none']",Семченко,И.В.,['Белгородский государственный университет'],none,none,...,4,0,0.0,33649676,NaN,1,1,http://elibrary.ru/item.asp?id=15488853,NaN,"Семченко, И.В. ПРОБЛЕМЫ СОЦИАЛЬНОЙ АДАПТАЦИИ Г..."
3,3,15500786,4,77952.0,"[['210'], 'none']",Хагуров,Темыр Айтечевич,['Кубанский государственный университет'],none,none,...,4,0,0.0,33651468,NaN,1,1,http://elibrary.ru/item.asp?id=15500786,NaN,"Хагуров, Т.А. Современная культура и воспитани..."
4,4,15501422,4,746025.0,"[['2541'], 'none']",Елишев,С.О.,['МГУ им. М.В. Ломоносова?'],none,none,...,4,22,3.0,33651490,NaN,0,1,http://elibrary.ru/item.asp?id=15501422,NaN,"Елишев, С.О. Формирование ценностных ориентаци..."


In [17]:
# 2. Получаем спиисок с id авторов в наших данных
author_ids = [] # создаем пустой список

for i in range(1, 8+1):
    author_ids.extend(list(df[f'author_{i}_id'])) # расширяем наш список за счет всех 8 колонок с авторами
    
print('Всего собрано id авторов', len(author_ids)) # всего собрано id авторов
author_ids = list(np.unique(author_ids))
print('Всего уникальных id авторов в наших данных', len(author_ids)) # id уникальных авторов по всем 8 колонкам

Всего собрано id авторов 601856
Всего уникальных id авторов в наших данных 24983


In [18]:
author_ids[0:5] # так выглядят наши данные, их нужно обработать - убрать ".0" на конце когда это подстрока присутствует

['0', '0.0', '1000010', '1000056', '1000056.0']

In [19]:
author_ids.remove('nan')
author_ids.remove('none')
author_ids = [int(i.replace('.0', '')) for i in author_ids]
author_ids.remove(0)
author_ids.remove(0)
author_ids = list(np.unique(author_ids))

Получаем список с id авторов. id были собраны по всем 8 колонкам `author_{}_id`, удалены значения `0`, `nan`, `none`, значения обновлены до уникальных.

In [20]:
author_ids[0:10]

[296, 346, 571, 619, 651, 904, 1269, 1270, 1277, 1278]

In [21]:
len(author_ids) # количество уникальных id авторов по всем восьми колонкам

19723

**Пример тестовой выгрузки**: пробую выгрузку из сервиса 011. Произвожу тестовую выгрузку. Определила sid как 011 (сервис 011 для получения данных по авторам), в качестве аргумента нужно

In [25]:
# Сейчас просто выгружаем данные!
counter = 0
test_id = ['296'] # случайный id для теста. Обязательно нужно заключить в скобки, иначе данные не выгрузятся
for i in test_id:
    url = "http://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid="+i
    response = requests.get(url)
    data = xmltodict.parse(response.content)
    tree_response = urlopen(url).read()
    tree_data = ET.fromstring(tree_response)


In [26]:
data # вот так выглядит объект data

OrderedDict([('root',
              OrderedDict([('request',
                            OrderedDict([('remoteAddr', '92.242.58.41'),
                                         ('authorid', '296')])),
                           ('items',
                            OrderedDict([('@numOfItems', '252'),
                                         ('@limit', '683000'),
                                         ('item',
                                          [OrderedDict([('@id', '30817242'),
                                                        ('@typeCode', 'RAR'),
                                                        ('@yearPubl', '1970'),
                                                        ('@retracted', '0'),
                                                        ('#text',
                                                         '30817242')]),
                                           OrderedDict([('@id', '39931825'),
                                                        ('@t

In [27]:
data['root'] # так выглядит объект root в data

OrderedDict([('request',
              OrderedDict([('remoteAddr', '92.242.58.41'),
                           ('authorid', '296')])),
             ('items',
              OrderedDict([('@numOfItems', '252'),
                           ('@limit', '683000'),
                           ('item',
                            [OrderedDict([('@id', '30817242'),
                                          ('@typeCode', 'RAR'),
                                          ('@yearPubl', '1970'),
                                          ('@retracted', '0'),
                                          ('#text', '30817242')]),
                             OrderedDict([('@id', '39931825'),
                                          ('@typeCode', 'COA'),
                                          ('@yearPubl', '1975'),
                                          ('@retracted', '0'),
                                          ('#text', '39931825')]),
                             OrderedDict([('@id', '40082461'),

In [28]:
data['root']['items'] # так выглядит часть root в items

OrderedDict([('@numOfItems', '252'),
             ('@limit', '683000'),
             ('item',
              [OrderedDict([('@id', '30817242'),
                            ('@typeCode', 'RAR'),
                            ('@yearPubl', '1970'),
                            ('@retracted', '0'),
                            ('#text', '30817242')]),
               OrderedDict([('@id', '39931825'),
                            ('@typeCode', 'COA'),
                            ('@yearPubl', '1975'),
                            ('@retracted', '0'),
                            ('#text', '39931825')]),
               OrderedDict([('@id', '40082461'),
                            ('@typeCode', 'COA'),
                            ('@yearPubl', '1980'),
                            ('@retracted', '0'),
                            ('#text', '40082461')]),
               OrderedDict([('@id', '30973257'),
                            ('@typeCode', 'RAR'),
                            ('@yearPubl', '1981'),


In [29]:
# продолжаем уточнять
data['root']['items']['item'] # так выглядят итоговые объекты в которые забиты статьи одного автора

[OrderedDict([('@id', '30817242'),
              ('@typeCode', 'RAR'),
              ('@yearPubl', '1970'),
              ('@retracted', '0'),
              ('#text', '30817242')]),
 OrderedDict([('@id', '39931825'),
              ('@typeCode', 'COA'),
              ('@yearPubl', '1975'),
              ('@retracted', '0'),
              ('#text', '39931825')]),
 OrderedDict([('@id', '40082461'),
              ('@typeCode', 'COA'),
              ('@yearPubl', '1980'),
              ('@retracted', '0'),
              ('#text', '40082461')]),
 OrderedDict([('@id', '30973257'),
              ('@typeCode', 'RAR'),
              ('@yearPubl', '1981'),
              ('@retracted', '0'),
              ('#text', '30973257')]),
 OrderedDict([('@id', '30812715'),
              ('@typeCode', 'RAR'),
              ('@yearPubl', '1982'),
              ('@retracted', '0'),
              ('#text', '30812715')]),
 OrderedDict([('@id', '22760050'),
              ('@typeCode', 'PRC'),
              ('@ye

In [30]:
data['root']['items']['item'][0] # далее с помощью обычной индексаации можно получить каждую из статей.
# в дальнейшем будем пользоваться циклом for

OrderedDict([('@id', '30817242'),
             ('@typeCode', 'RAR'),
             ('@yearPubl', '1970'),
             ('@retracted', '0'),
             ('#text', '30817242')])

In [31]:
data['root']['items']['item'][0]

OrderedDict([('@id', '30817242'),
             ('@typeCode', 'RAR'),
             ('@yearPubl', '1970'),
             ('@retracted', '0'),
             ('#text', '30817242')])

In [33]:
for i in range(len(data['root']['items']['item'])):
    print(data['root']['items']['item'][i]) # вот такие словари у нас получаются. Это все статьи автора

OrderedDict([('@id', '30817242'), ('@typeCode', 'RAR'), ('@yearPubl', '1970'), ('@retracted', '0'), ('#text', '30817242')])
OrderedDict([('@id', '39931825'), ('@typeCode', 'COA'), ('@yearPubl', '1975'), ('@retracted', '0'), ('#text', '39931825')])
OrderedDict([('@id', '40082461'), ('@typeCode', 'COA'), ('@yearPubl', '1980'), ('@retracted', '0'), ('#text', '40082461')])
OrderedDict([('@id', '30973257'), ('@typeCode', 'RAR'), ('@yearPubl', '1981'), ('@retracted', '0'), ('#text', '30973257')])
OrderedDict([('@id', '30812715'), ('@typeCode', 'RAR'), ('@yearPubl', '1982'), ('@retracted', '0'), ('#text', '30812715')])
OrderedDict([('@id', '22760050'), ('@typeCode', 'PRC'), ('@yearPubl', '1986'), ('@retracted', '0'), ('#text', '22760050')])
OrderedDict([('@id', '31020309'), ('@typeCode', 'RAR'), ('@yearPubl', '1988'), ('@retracted', '0'), ('#text', '31020309')])
OrderedDict([('@id', '22472030'), ('@typeCode', 'RAR'), ('@yearPubl', '1989'), ('@retracted', '0'), ('#text', '22472030')])
OrderedD

In [34]:
for j in data['root']['items']['item'][0]: # берем первую статью
    print(j) # вот какие объекты забиты внутри одной записи про одну статью

@id
@typeCode
@yearPubl
@retracted
#text


In [35]:
print(data['root']['items']['item'][0]['@id']) # так получается id одной статьи
print(data['root']['items']['item'][0]['@typeCode']) # так получается тип
print(data['root']['items']['item'][0]['@yearPubl']) # так получается год
print(data['root']['items']['item'][0]['@retracted']) # так получается статус (отозвана?)
print(data['root']['items']['item'][0]['#text']) # так получается текст

30817242
RAR
1970
0
30817242


## Тестовая выгрузка с получением всех данных

Перед тем, как начать тестовую выгрузку, проверяю структуру данных из конца списка - беру последний айдишник автора и открываю его страницу. Я предварительно смотрела на тестовую выгрузку из пяти авторов из начала списка, и меня очень смутило количество их публикаций – сотни штук. Проверив автора с id `1158945`, я успокоилась: у него всего две публикации, за 2018 и 2022 год. Логиично, что у авторов из начала будет больше публикаций – они как минимум старше и раньше начали свою академическую деятельность.

На всякий случай, я перепроверила данные вручную. Алгоритм следующий:
1. Открывааем [Elibrary](https://www.elibrary.ru);
2. Входим в свой аккаунт;
3. В правом меню снизу выбираем `Навигатор` -> `Авторы`;
4. Открывается поисковое окно. В маленьком окошке выбираем `РИНЦ Author id` и вписываем в окошко справа значение id. Нажимаем на `Поиск`;
5. Нам показывают автора. Нажимаем на публикации (число публикаций). Открывается окно со всеми статьями, включенными в РИНЦ. Нажимаем на первую статью автора, далее - на имя и выбираем `Профиль автора в SCIENCE INDEX`;
6. Открывается страница со всеми публикациями автора в Elibrary. 

Для автора с id `1158945` открывается в результате манипуляций выше следующая ссылка: https://www.elibrary.ru/author_items.asp?authorid=1158945&show_option=1&show_refs=1 , на которой действительно находится две статьи - за 2018 и 2022 год.

In [49]:
author_ids[-1] # проверяем на авторе из конца списка
# ссылка для просмотра страницы
print(f'https://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid={author_ids[-1]}')

https://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid=1158945


Тестовые ссылки:

1. Автор 296: https://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid=296



2. Автор 346: https://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid=346



3. Автор 571: https://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid=571



4. Автор 619: https://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid=619



5. Автор 651: https://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid=651



In [62]:
counter = 0
test_id = author_ids[0:5] # 5 первых id для теста

one_row_one_paper = pd.DataFrame()

for j in tqdm.tqdm(test_id):
    author_id = j
    url = "http://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid="+str(j)
    response = requests.get(url)
    data = xmltodict.parse(response.content) # с этим объектом будем работать

    # создаем объекты с которыми будем работать, это все для записи данных по одному автору
    author_paper_ids = [] # айдишники статей одного автора
    author_types = [] # тип статей одного автора
    author_years = [] # год статей одного автора
    author_retracteds = [] # статус (отозвана или нет(?)) одного автора
    author_texts = [] # текст чего-то там одного автора
    
    # идем по всем статьям автора
    for i in range(len(data['root']['items']['item'])):
        
        ## ID статьи
        try:
            this_paper_id = data['root']['items']['item'][i]['@id']
        except:
            this_paper_id = 'none'
        author_paper_ids.append(this_paper_id) # добавляем id статьи к списку всех статей автора
        
        ## ТИП статьи 
        try:
            this_paper_type = data['root']['items']['item'][0]['@typeCode'] # так получается тип
        except:
            this_paper_type = 'none'
        author_types.append(this_paper_type) # добавляем тип статьи к списку всех типов статей аавтора
        
        ## ГОД статьи
        try:
            this_paper_year = data['root']['items']['item'][0]['@yearPubl']
        except:
            this_paper_year = 'none'
        author_years.append(this_paper_year) # добавляем год статьи к списку всех годов статей автора
        
        ## RETRACTED или нет?
        try:
            this_paper_retracted = data['root']['items']['item'][0]['@retracted'] # так получается статус (отозвана?)
        except:
            this_paper_retracted = 'none'
        author_retracteds.append(this_paper_retracted) # добавляем статус стаатьи (retracted или нет) к списку со статусами всех статей автора
        
        ## ТЕКСТ (не знаю что это означает) статьи
        try:
            this_paper_text = data['root']['items']['item'][0]['#text']
        except:
            this_paper_text = 'none'
        author_texts.append(this_paper_text) # добавляем текст статьи к списку с текстами (still dont know what does it means) статей автора
            
    author_personal_id = [str(author_id) for i in range(len(data['root']['items']['item']))]

    # получаем датафрейм где только одна строчка - данные по одной статье одного автора
    one_author = pd.DataFrame([author_personal_id,
              author_paper_ids,
              author_types,
              author_years,
              author_retracteds,
              author_texts]).transpose() 
            
    # теперь обновляем наш датафрейм где данные в формате одна строчка - один автор        
    one_row_one_paper = one_row_one_paper.append(one_author)
    
    
            
    counter += 1 #обновляем счетчик
    if counter % 50 == 0: # каждые 50 итерациий
        sleep(1) # засыпаем на 1 секунду

100%|██████████| 5/5 [00:00<00:00,  7.14it/s]


In [63]:
one_row_one_paper.columns = ['author_id',
                             'paper_id',
                             'paper_type',
                             'paper_year',
                             'paper_retracted',
                             'paper_text']

In [67]:
one_row_one_paper # так выглядит текстовая выгрузка

,author_id,paper_id,paper_type,paper_year,paper_retracted,paper_text
0,296,30817242,RAR,1970,0,30817242
1,296,39931825,RAR,1970,0,30817242
2,296,40082461,RAR,1970,0,30817242
3,296,30973257,RAR,1970,0,30817242
4,296,30812715,RAR,1970,0,30817242
...,...,...,...,...,...,...
181,651,20473633,COR,1966,0,30945032
182,651,21896365,COR,1966,0,30945032
183,651,21897522,COR,1966,0,30945032
184,651,25557109,COR,1966,0,30945032


## Основная выгрузка по всем авторам

Это третий этап - основная выгрузка всех данных о статьях интересующих нас авторов. **Запустите ячейку с кодом ниже**.

**Важно! На данном этапе у вас могут возникнуть проблемы из-за стабильности сети. Что следует делать в этом случае?**
1. Создать новую ячейку;
2. Скопировать туда код из ячейки ниже; 
3. Поменять `one_row_one_paper` на другое название, например, на `second_part`, и заменить на него же в остальных частях кода;
4. Создать список `needed_author_ids`, куда вы сохраните все id авторов из изначального списка со всеми id авторов `author_ids`, если их нет в `one_row_one_paper['author_id']`:

>`needed_author_ids = [i for i in author_ids if i not in one_row_one_paper['author_id'].tolist()]`

5. Поменяете в вашей новой ячейке кода `author_ids` на `needed_author_ids` (чтобы во второй итерации выгрузить данные только по тем авторам, по которым еще не выгрузили данные);
6. Запустить ячейку с кодом и молиться на хорошее качество интернет-соединения!

In [ ]:
counter = 0 # заводим пока пустой счетчик;

one_row_one_paper = pd.DataFrame() # создаем pandas.DataFrame, куда мы будем сохранять данные в формате "одна строчка - одна статья одного автора"

# далее мы идем по списку с id наших авторов из изначальных данных
for j in tqdm.tqdm(author_ids):
    author_id = j
    url = "http://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=011&authorid="+str(j)
    response = requests.get(url) # осуществляем запрос к Service-011 API Elibrary;
    data = xmltodict.parse(response.content) # с этим объектом будем работать;

    
    # создаем объекты с которыми будем работать, это все для записи данных по одному автору
    author_paper_ids = [] # айдишники статей одного автора;
    author_types = [] # тип статей одного автора;
    author_years = [] # год статей одного автора;
    author_retracteds = [] # статус (отозвана или нет(?)) одного автора;
    author_texts = [] # текст чего-то там одного автора;
    
    # идем по всем статьям автора
    for i in range(len(data['root']['items']['item'])):
        
        ## ID статьи
        try:
            this_paper_id = data['root']['items']['item'][i]['@id']
        except:
            this_paper_id = 'none'
        author_paper_ids.append(this_paper_id) # добавляем id статьи к списку всех статей автора
        
        ## ТИП статьи 
        try:
            this_paper_type = data['root']['items']['item'][0]['@typeCode'] # так получается тип
        except:
            this_paper_type = 'none'
        author_types.append(this_paper_type) # добавляем тип статьи к списку всех типов статей аавтора
        
        ## ГОД статьи
        try:
            this_paper_year = data['root']['items']['item'][0]['@yearPubl']
        except:
            this_paper_year = 'none'
        author_years.append(this_paper_year) # добавляем год статьи к списку всех годов статей автора
        
        ## RETRACTED или нет?
        try:
            this_paper_retracted = data['root']['items']['item'][0]['@retracted'] # так получается статус (отозвана?)
        except:
            this_paper_retracted = 'none'
        author_retracteds.append(this_paper_retracted) # добавляем статус стаатьи (retracted или нет) к списку со статусами всех статей автора
        
        ## ТЕКСТ (не знаю что это означает) статьи
        try:
            this_paper_text = data['root']['items']['item'][0]['#text']
        except:
            this_paper_text = 'none'
        author_texts.append(this_paper_text) # добавляем текст статьи к списку с текстами (still dont know what does it means) статей автора
            
    author_personal_id = [str(author_id) for i in range(len(data['root']['items']['item']))]

    # получаем датафрейм где только одна строчка - данные по одной статье одного автора
    one_author = pd.DataFrame([author_personal_id,
              author_paper_ids,
              author_types,
              author_years,
              author_retracteds,
              author_texts]).transpose() 
            
    # теперь обновляем наш датафрейм где данные в формате одна строчка - один автор (присоединяем новые данные)       
    one_row_one_paper = one_row_one_paper.append(one_author)
    
    counter += 1 #обновляем счетчик
    if counter % 50 == 0: # каждые 50 итерациий
        sleep(1) # засыпаем на 1 секунду

Получаем финальный файл (датафрейм `final`), содержащий в себе информацию о всех статьях авторов, которые были упомянуты в наших изначальных авторов

In [ ]:
# добавляем названия колонкам
one_row_one_paper.columns = ['author_id',
                             'paper_id',
                             'paper_type',
                             'paper_year',
                             'paper_retracted',
                             'paper_text']

In [24]:
final = one_row_one_paper
final # получаем финальные данные; одна строчка - одна статья одного автора; 
# самое главное - тут есть все id всех статей авторов из нашего изначального набора данных;
# при желании, можно выгрузить данные и по ним используя код из нашего jupyter-notebook под цифрой 1 в нашем репозитории

,author_id,paper_id,paper_type,paper_year,paper_retracted,paper_text
0,296,30817242,RAR,1970,0,30817242
1,296,39931825,RAR,1970,0,30817242
2,296,40082461,RAR,1970,0,30817242
3,296,30973257,RAR,1970,0,30817242
4,296,30812715,RAR,1970,0,30817242
...,...,...,...,...,...,...
6,1158931,43149292,PRC,2016,0,27464636
7,1158931,46203919,PRC,2016,0,27464636
8,1158931,46515617,PRC,2016,0,27464636
0,1158945,36736120,RAR,2018,0,36736120


In [36]:
final['paper_year'].value_counts() # Для интереса можем посмотреть на количество статей по годам

2006    58829
2004    57999
2003    56808
2000    54151
2005    53293
        ...  
2021      119
1949      117
1955      113
1956       40
1931       11
Name: paper_year, Length: 70, dtype: int64

## Сохранение файлов

Это финальный, 4 шаг. Мы сохраняем данные – тут записаны все статьи авторов из нашего изначального датафрейма. В нашем случае этот `pandas.DataFrame` имеет размерность в 1 миллион 143 тысячи 505 строк. Из-за такой размерности `.xlsx` файл не сохранился, сохранился только `.csv` файл.

In [26]:
final.to_csv('all_final_authors_with_papers.csv')